<a href="https://colab.research.google.com/github/alemando/redesNeuronales/blob/main/notebooks/9-optimizacion_usando_ElasticNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimización usando ElasticNet
===

En esta técnica se combina la penalización de ridge regression y LASSO, tal como se indica en la siguiente ecuación:

$$ \sum_{i=1}^N (y_i - g(x_i))^2 + \alpha \rho * (\sum_{p=1} |w_p|) + \frac{\alpha (1 - \rho)}{2}(\sum_{p=1} w_p^2)$$

$\alpha$ y $\rho$ son hiperparámetros suministrados por el usuario. **Nota:** Para el modelo lineal, la penalización solo aplica para los coeficientes de $x$, no para el intercepto.

In [6]:
#
# A continuación se presenta la implementación de un modelo de
# regresión lineal que usa la función de penalización ElasticNet 
# para estimar los parámetros óptimos. Complete el código presentado
# para que pasen las pruebas definidas en las celdas restantes.
#
import numpy as np
import pandas as pd
import pytest


class ElasticNetRegression:
    def __init__(self, intercept, coef, maxiter, mu, alpha, rho):
        self.intercept_ = intercept
        self.coef_ = np.array(coef)
        self._maxiter = maxiter
        self._mu = mu
        self._alpha = alpha
        self._rho = rho

    def compute_loss(self, x, y):
        d = self.predict(x)
        error = np.power(np.array(y) - np.array(d),2)
        loss = (sum(error) + self._alpha * self._rho * sum(abs(self.coef_)) + 
                self._alpha*(1 - self._rho)*sum(np.power(self.coef_,2))/2)
        return loss

    def predict(self, x):
        return [self.coef_[0]*d[0] + self.coef_[1]*d[1] + self.intercept_ for d in x]

    def compute_gradient(self, x, y):
        grd = []

        error = np.array([yi - di for yi, di in zip(y, self.predict(x))])

        self._grad_intercept = -2*np.sum(error)
        
        gSSE1 = -2*np.sum([e*i[0] for e, i in zip(error,x)])
        PenLasso = self._alpha * self._rho * (1 if self.coef_[0] > 0 else -1)
        PenElastic = self._alpha * (1 - self._rho) * self.coef_[0]
        grd.append(gSSE1 + PenLasso + PenElastic)
 
        gSSE2 = -2*np.sum([e*i[1] for e, i in zip(error,x)])
        PenLasso = self._alpha * self._rho * (1 if self.coef_[1] > 0 else -1)
        PenElastic = self._alpha * (1 - self._rho) * self.coef_[1]
        grd.append(gSSE2 + PenLasso + PenElastic) 
 
        self._grad_coef = np.array(grd)

    def fit(self, x, y):
        for iter in range(self._maxiter):
            self.compute_gradient(x, y)
            self.improve()

    def improve(self):
        self.intercept_ = self.intercept_ - self._mu * self._grad_intercept
        self.coef_ = self.coef_ - self._mu * self._grad_coef


x = [
    [0.0, 0.1],
    [0.2, 0.3],
    [0.4, 0.5],
    [0.6, 0.7],
    [0.8, 0.9],
    [1.0, 1.1],
]

# y = 1 x1 + 1.1 x2 + 0.2
y = [
    0.31,
    0.73,
    1.15,
    1.57,
    1.99,
    2.41,
]

In [7]:
#
# Test 1
# =============================================================================
# Implemente la función de pérdida.
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = ElasticNetRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=10,
    rho = 10,
)
pytest.approx(lr.compute_loss(x, y), 0.0001) == 51.7044

True

In [8]:
#
# Test 2
# =============================================================================
# Implemente la función de pronóstico
#
# Rta/
# True
#

# ---->>> Evaluación ---->>>
lr = ElasticNetRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=10,
    rho = 10,
)
all(
    pytest.approx(a) == b
    for a, b in zip(lr.predict(x), [0.13, 0.23, 0.33, 0.43, 0.53, 0.63])
)

True

In [9]:
#
# Test 3
# =============================================================================
# Implemente el gradiente
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = ElasticNetRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=10,
    rho = 10,
)
lr.compute_gradient(x, y)
print(lr._grad_intercept == pytest.approx(-11.76))
print(all(pytest.approx(a) == b for a, b in zip(lr._grad_coef, [73.88 , 63.704])))

True
True


In [10]:
#
# Test 4
# =============================================================================
# Implemente la función fit
#
# Rta/
# True
# True
#

# ---->>> Evaluación ---->>>
lr = ElasticNetRegression(
    intercept=0.1,
    coef=[0.2, 0.3],
    maxiter=10000,
    mu=0.001,
    alpha=10,
    rho = 10,
)
lr.fit(x, y)
print(pytest.approx(lr.intercept_, 0.001) == 1.391750)
print(all(pytest.approx(a, 0.001) == b for a, b in zip(lr.coef_, [0.010658, 0.057296])))

True
True
